## Section 3: Distributed Memory Assembly of Matrix and Right-Hand Side Vector 
- using [PartitionedMatrices](https://github.com/fverdugo/PartitionedArrays.jl); need to study the example described at [example](https://www.francescverdugo.com/PartitionedArrays.jl/dev/usage/).  

<div>
<img src="./figures/dag-fem-1d.jpg" width=400 /> 
<center> Figure 1: Directed acyclic graph representation of decomposed mesh of 4 elements on the interval. A finite element discretization is assumed. </center>   
</div>

Following [Au = f example here.](https://www.francescverdugo.com/PartitionedArrays.jl/dev/examples/#Distributed-sparse-linear-solve)

In [1]:
using PartitionedArrays
using Plots
using IterativeSolvers
using LinearAlgebra

In [2]:
#..first generate the row partition
np = 3
N = 10
ranks = LinearIndices((np,))
row_partition = uniform_partition(ranks,N+1)

3-element Vector{PartitionedArrays.LocalIndicesWithConstantBlockSize{1}}:
 [1, 2, 3]
 [4, 5, 6, 7]
 [8, 9, 10, 11]

In [3]:
#..construct the mesh: see before 
h = 1/N; 
x = Vector(0:h:1); 

#..Mesh with points and edges 
#..point holds the coordinates of the left and right node of the element
#..edges holds the global indices of the left and right node of the element
points = collect( [x[i], x[i+1]] for i in 1:length(x)-1) 
edges = collect( [i, i+1] for i in 1:length(x)-1); 

#..Set the source function 
fsource(x) = x*(x-1); 

Something to compare with

In [4]:
#..Initialize global matrix and right-hand side value 
A = zeros(length(x), length(x)); 
f = zeros(length(x), 1); 

#..Perform loop over elements and assemble global matrix and vector 
for i=1:length(edges) 

  xl, xr = points[i,:][1]
  floc = (xr-xl) * [fsource(xl) fsource(xr)];
  Aloc = (1/(xr-xl))*[1 -1; -1 1]; 

  for j=1:2 
    f[edges[i][j]] += floc[j];
    for k =1:2 
      A[edges[i][j], edges[i][k]] += Aloc[j,k]; 
    end 
  end 

end

f_orig = copy(f)
A_orig = copy(A)
f

11×1 Matrix{Float64}:
  0.0
 -0.018000000000000002
 -0.032
 -0.041999999999999996
 -0.048
 -0.04999999999999999
 -0.04799999999999999
 -0.04200000000000002
 -0.032
 -0.01799999999999999
  0.0

In [5]:
A

11×11 Matrix{Float64}:
  10.0  -10.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
 -10.0   20.0  -10.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
   0.0  -10.0   20.0  -10.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
   0.0    0.0  -10.0   20.0  -10.0    0.0    0.0    0.0    0.0    0.0    0.0
   0.0    0.0    0.0  -10.0   20.0  -10.0    0.0    0.0    0.0    0.0    0.0
   0.0    0.0    0.0    0.0  -10.0   20.0  -10.0    0.0    0.0    0.0    0.0
   0.0    0.0    0.0    0.0    0.0  -10.0   20.0  -10.0    0.0    0.0    0.0
   0.0    0.0    0.0    0.0    0.0    0.0  -10.0   20.0  -10.0    0.0    0.0
   0.0    0.0    0.0    0.0    0.0    0.0    0.0  -10.0   20.0  -10.0    0.0
   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  -10.0   20.0  -10.0
   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  -10.0   10.0

Compute the rhs vector

In [6]:
j = 2
IV = map(row_partition) do row_indices
    I,V = Int[], Float64[]
    for global_row in local_to_global(row_indices) 
        # xl, xr = points[global_row,:][1]
        # floc = (xr-xl) * [fsource(xl) fsource(xr)]
        # f[edges[global_row][j]] += floc[j]
        # xlp1, xrp1 = points[global_row+1,:][1]
        # floc = (xrp1-xlp1) * [fsource(xlp1) fsource(xrp1)]
        # f[edges[global_row][2]] += floc[2]
        if global_row == 1
            xl, xr = points[global_row,:][1]
            floc = (xr-xl) * [fsource(xl) fsource(xr)]
            v = floc[1]
        elseif global_row == N+1
            xl, xr = points[global_row-1,:][1]
            floc = (xr-xl) * [fsource(xl) fsource(xr)]
            v = floc[2]
        else
            xll, xrl = points[global_row-1,:][1]
            flocl = (xrl-xll) * [fsource(xll) fsource(xrl)]
            xlr, xrr = points[global_row,:][1]
            flocr = (xrr-xlr) * [fsource(xlr) fsource(xrr)]
            v = flocr[1] + flocl[2]
        end
        push!(I,global_row)
        push!(V,v)
    end
    I,V
end
I,V = tuple_of_arrays(IV)
f = pvector!(I,V,row_partition) |> fetch
own_values(f)
#..handle the boundary conditions in the right-hand side vector 
# right-hand side vector already satisfied:
# f[1]   = 0;     f[end] = 0;

3-element Vector{SubArray{Float64, 1, Vector{Float64}, Tuple{UnitRange{Int32}}, true}}:
 [0.0, -0.018000000000000002, -0.032]
 [-0.041999999999999996, -0.048, -0.04999999999999999, -0.04799999999999999]
 [-0.04200000000000002, -0.032, -0.01799999999999999, 0.0]

Compute the system matrix

In [7]:
#  xl, xr = points[i,:][1]
#  Aloc = (1/(xr-xl))*[1 -1; -1 1];

#  for j=1:2 
#    for k =1:2 
#      A[edges[i][j], edges[i][k]] += Aloc[j,k]; 
#    end 
#  end 

IJV = map(row_partition) do row_indices
    I,J,V = Int[], Int[], Float64[]
    for global_row in local_to_global(row_indices)
        if global_row == 1
            # not needed due to boundary condition: xl, xr = points[global_row,:][1]
            # not needed due to boundary condition: Aloc = (1/(xr-xl))*[1 -1; -1 1]
            # not needed due to boundary condition: push!(I,global_row)
            # not needed due to boundary condition: push!(J,global_row)
            # not needed due to boundary condition: push!(V,Aloc[1,1])
            # not needed due to boundary condition: push!(I,global_row)
            # not needed due to boundary condition: push!(J,global_row+1)
            # not needed due to boundary condition: push!(V,Aloc[1,2])
#..handle the boundary conditions in the matrix
# A[1,1] = 1;     A[1,2] = 0
            push!(I,global_row)
            push!(J,global_row)
            push!(V,1.0)
            push!(I,global_row)
            push!(J,global_row+1)
            push!(V,0.0)
        elseif global_row == N+1
            # not needed due to boundary condition: xl, xr = points[global_row-1,:][1]
            # not needed due to boundary condition: Aloc = (1/(xr-xl))*[1 -1; -1 1]
            # not needed due to boundary condition: push!(I,global_row)
            # not needed due to boundary condition: push!(J,global_row-1)
            # not needed due to boundary condition: push!(V,Aloc[2,1])
            # not needed due to boundary condition: push!(I,global_row)
            # not needed due to boundary condition: push!(J,global_row)
            # not needed due to boundary condition: push!(V,Aloc[2,2])
#..handle the boundary conditions in the matrix
# A[end,end-1]=0; A[end,end] = 1
            push!(I,global_row)
            push!(J,global_row-1)
            push!(V,0.0)
            push!(I,global_row)
            push!(J,global_row)
            push!(V,1.0)
        else
            xll, xrl = points[global_row-1,:][1]
            Alocl = (1/(xrl-xll))*[1 -1; -1 1]
            xlr, xrr = points[global_row,:][1]
            Alocr = (1/(xrr-xlr))*[1 -1; -1 1]
            push!(I,global_row)
            push!(J,global_row-1)
            push!(V,Alocl[2,1])
            push!(I,global_row)
            push!(J,global_row)
            push!(V,Alocl[2,2] + Alocr[1,1])
            push!(I,global_row)
            push!(J,global_row+1)
            push!(V,Alocr[1,2])
        end
    end
    I,J,V
end

I,J,V = tuple_of_arrays(IJV)
col_partition = row_partition
A = psparse!(I,J,V,row_partition,col_partition) |> fetch;

In [8]:
V

3-element Vector{Vector{Float64}}:
 [1.0, 0.0, -10.0, 20.0, -10.0, -10.0, 20.0, -10.000000000000002]
 [-10.000000000000002, 20.0, -9.999999999999996, -9.999999999999996, 20.0, -10.000000000000002, -10.000000000000002, 20.000000000000004, -10.000000000000002, -10.000000000000002, 20.000000000000004, -10.000000000000002]
 [-10.000000000000002, 19.999999999999993, -9.999999999999991, -9.999999999999991, 19.999999999999993, -10.000000000000002, -10.000000000000002, 20.000000000000004, -10.000000000000002, 0.0, 1.0]

Generate an initial guess that fulfills the boundary conditions. Solve and check the result

In [9]:
u = similar(f,axes(A,2))
u .= f
IterativeSolvers.cg!(u,A,f)
r = A*u - f
norm(r)

4.76123924537016e-16

In [10]:
own_values(u)

3-element Vector{SubArray{Float64, 1, Vector{Float64}, Tuple{UnitRange{Int32}}, true}}:
 [0.0, -0.016499999999999997, -0.031200000000000012]
 [-0.04270000000000001, -0.04999999999999999, -0.0525, -0.049999999999999996]
 [-0.042699999999999995, -0.03119999999999998, -0.0165, 0.0]

In [11]:
#..plot the solution  
p1=plot(x,u,shape=:circle,lw=2,legend=false)
xlabel!("x") 
ylabel!("u(x)")
title!("Numerically computed solution")

LoadError: Scalar indexing on PVector is not allowed for performance reasons.